In [1]:
# from mainnet_launch.database.schema.full import *
# from mainnet_launch.database.schema.postgres_operations import *
# from mainnet_launch.constants import *

# autopool = BASE_USD
# # note this is old data, we need to fetch it live,
# destination_states_df = merge_tables_as_df(
#     [
#         TableSelector(DestinationTokenValues, select_fields=[DestinationTokenValues.destination_vault_address, DestinationTokenValues.token_address, DestinationTokenValues.quantity, DestinationTokenValues.spot_price]),

#         TableSelector(
#             DestinationStates,
#             select_fields=[
#                 DestinationStates.underlying_token_total_supply,
#                 DestinationStates.block,
#             ],
#             join_on=(DestinationTokenValues.destination_vault_address == DestinationStates.destination_vault_address )
#         ),
#         TableSelector(
#             Destinations,
#             select_fields=[
#                 Destinations.underlying_name,
#             ],
#             join_on=Destinations.destination_vault_address == DestinationStates.destination_vault_address,
#         ),
#     ],
#     where_clause=DestinationStates.chain_id == autopool.chain.chain_id,
# )


# autopool_destination_states_df = merge_tables_as_df(
#     [
#         TableSelector(
#             AutopoolDestinationStates,
#             select_fields=[AutopoolDestinationStates.autopool_vault_address, AutopoolDestinationStates.destination_vault_address, AutopoolDestinationStates.owned_shares, AutopoolDestinationStates.block],
#         ),
#         TableSelector(
#             Autopools,
#             select_fields=[Autopools.name],
#             join_on=Autopools.autopool_vault_address == AutopoolDestinationStates.autopool_vault_address,
#         ),
#     ],
#     where_clause=(AutopoolDestinationStates.autopool_vault_address == autopool.autopool_eth_addr)
# )

# highest_block = autopool_destination_states_df['block'].max()
# latest_destination_states_df = destination_states_df[destination_states_df['block'] == highest_block].copy()
# latest_autopool_destination_states_df = autopool_destination_states_df[autopool_destination_states_df['block'] == highest_block].copy()
# latest_destination_states_df

In [2]:
# df = pd.merge(
#     latest_autopool_destination_states_df,
#     latest_destination_states_df,
#     left_on="destination_vault_address",
#     right_on="destination_vault_address",
# )
# df["percent_ownership"] = 100 * df["owned_shares"] / df["underlying_token_total_supply"]
# percent_ownership_by_destination = (
#     df.groupby(["name", "underlying_name"])["percent_ownership"]
#     .sum()
#     .reset_index()
#     .sort_values(by="percent_ownership", ascending=False)
# )
# print(highest_block, autopool.name)
# percent_ownership_by_destination

In [3]:
# autoUSD has not updated in the db since July 7th 2025, not sure why it is broken not updating

In [4]:
df

NameError: name 'df' is not defined

In [ ]:
autopool_destination_states_df

,autopool_vault_address,destination_vault_address,owned_shares,name


In [ ]:
df["block"].value_counts().sort_index()

Series([], Name: count, dtype: int64)

In [ ]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [7]:
# https://github.com/Tokemak/v2-swap-routes-watcher/blob/main/src/core/helpers/aggregator/coingecko.ts
# based on this
from pprint import pprint

from mainnet_launch.constants import ChainData, eth_client, COINGECKO_API_KEY, ETH_CHAIN, BASE_CHAIN, SONIC_CHAIN
from web3 import Web3
import pandas as pd
import requests
import asyncio
import aiohttp
import plotly.express as px


def fetch_coingeck_pools(start_token: str, chain: ChainData, min_USD_reserves: int = 100_000):
    if chain == ETH_CHAIN:
        coingecko_coin_slug = "eth"
    elif chain == BASE_CHAIN:
        coingecko_coin_slug = "base"
    elif chain == SONIC_CHAIN:
        coingecko_coin_slug = "sonic"

    base_url = "https://pro-api.coingecko.com/api/v3"
    url = f"{base_url}/onchain/networks/{coingecko_coin_slug}/tokens/{start_token}/pools"
    headers = {"x-cg-pro-api-key": COINGECKO_API_KEY}
    params = {"sort": "h24_volume_usd_liquidity_desc", "include": ["base_token", "quote_token"]}

    resp = requests.get(url, headers=headers, params=params, timeout=30)
    resp.raise_for_status()
    pools = resp.json()["data"]

    def _extract_pool_fields_from_coingecko(pools):
        rows = []
        for p in pools:
            row = {
                "address": Web3.toChecksumAddress(p["attributes"]["address"]),
                "name": p["attributes"]["name"],
                "reserve_in_usd": round(float(p["attributes"]["reserve_in_usd"])),
                "base_token_id": p["relationships"]["base_token"]["data"]["id"].split("_")[1],
                "quote_token_id": p["relationships"]["quote_token"]["data"]["id"].split("_")[1],
                "dex_id": p["relationships"]["dex"]["data"]["id"],
            }

            rows.append(row)

        return pd.DataFrame.from_records(rows)

    df = _extract_pool_fields_from_coingecko(pools)

    return df[df["reserve_in_usd"] >= min_USD_reserves]


def fetch_n_hops_from_tokens_with_coingecko(
    tokens_to_check: set[str], chain: str, min_USD_reserves: int = 100_000, n_hops: int = 2
) -> pd.DataFrame:
    tokens_to_check = set([eth_client.toChecksumAddress(t).lower() for t in tokens_to_check])
    tokens_already_checked = set()

    dex_liqudity_df = None

    for hop_num in range(n_hops):
        tokens_to_check_this_hop = [t for t in tokens_to_check if t not in tokens_already_checked]
        print(f"Checking {len(tokens_to_check_this_hop)} tokens in this hop. {hop_num}")
        for token in tokens_to_check_this_hop:
            if dex_liqudity_df is None:
                dex_liqudity_df = fetch_coingeck_pools(token, chain, min_USD_reserves=min_USD_reserves)
            else:
                this_token_hops_df = fetch_coingeck_pools(token, chain, min_USD_reserves=min_USD_reserves)
                dex_liqudity_df = pd.concat([dex_liqudity_df, this_token_hops_df], ignore_index=True)

            tokens_already_checked.add(eth_client.toChecksumAddress(token))

        new_tokens_to_check = set(
            dex_liqudity_df["quote_token_id"].tolist() + dex_liqudity_df["base_token_id"].tolist()
        )
        new_tokens_to_check = set([eth_client.toChecksumAddress(t).lower() for t in new_tokens_to_check])
        tokens_to_check = new_tokens_to_check

    return dex_liqudity_df.drop_duplicates()


async def fetch_pair(session, chain: str, pool_address: str):
    def _extract_sided_liqudity_from_dex_screener(data: dict) -> dict:
        """
        Given a DexScreener response JSON (with 'pairs' or 'pair'),
        return a dict mapping each token address to its token liquidity,
        plus the total USD TVL under the key 'usd_tvl'.
        """
        # pick whichever key exists
        pair = data["pair"]
        if pair is None:
            return {"pairAddress": pool_address}

        data["pairAddress"] = pool_address
        return data

    # some of the dex screener endpoints have their liqudity in ETH terms, not USD terms
    url = f"https://api.dexscreener.com/latest/dex/pairs/{chain}/{pool_address.lower()}"
    async with session.get(url, timeout=30) as resp:
        resp.raise_for_status()
        data = await resp.json()
        return _extract_sided_liqudity_from_dex_screener(data)


async def get_dex_sided_liquidity(chain: str, pool_addresses: list[str]):
    results = []
    BATCH_SIZE = 200
    for i in range(0, len(pool_addresses), BATCH_SIZE):
        batch = pool_addresses[i : i + BATCH_SIZE]
        async with aiohttp.ClientSession() as session:
            # fire off up to 200 in parallel
            batch_results = await asyncio.gather(*(fetch_pair(session, chain, addr) for addr in batch))
            results.extend(batch_results)

        # if there’s more work to do, wait a minute
        if i + BATCH_SIZE < len(pool_addresses):
            await asyncio.sleep(60)

    return pd.DataFrame(results)


def _flatten_all_dicts(df: pd.DataFrame) -> pd.DataFrame:
    """
    For every column in df that contains dicts, json_normalize it
    (flattening all levels) and prefix the new columns with the
    original column name + '_'.
    """
    df_flat = df.copy()
    for col in df_flat.columns:
        # detect any dict in this column
        if df_flat[col].apply(lambda x: isinstance(x, dict)).any():
            # json_normalize the list of dicts
            exploded = pd.json_normalize(df_flat[col].tolist(), sep="_").add_prefix(col + "_")

            # drop the old nested column, reset index so join lines up
            df_flat = df_flat.drop(columns=[col]).reset_index(drop=True).join(exploded)
    return df_flat


async def fetch_sonic_tvl():

    scUSD = "0xd3DCe716f3eF535C5Ff8d041c1A41C3bd89b97aE"
    frxUSD = "0x80Eede496655FB9047dd39d9f418d5483ED600df"
    USDC = "0x29219dd400f2Bf60E5a23d13Be72B486D4038894"

    sonic_token_universe = set([scUSD, frxUSD, USDC])

    dex_liqudity_df = fetch_n_hops_from_tokens_with_coingecko(
        tokens_to_check=sonic_token_universe, chain=SONIC_CHAIN, min_USD_reserves=100_000, n_hops=2
    )

    sided_liquidity_df = await get_dex_sided_liquidity("sonic", dex_liqudity_df["address"].tolist())
    # full_df = pd.merge(sided_liquidity_df, dex_liqudity_df, left_on='pairAddress', right_on='address', how='left').dropna()

    df = _flatten_all_dicts(sided_liquidity_df)
    df = df.dropna(subset=["pair_liquidity_usd"])
    df = df.drop_duplicates(subset=["pairAddress"])

    df["base_token_usd_tvl"] = df["pair_liquidity_base"] * df["pair_priceUsd"].astype(float)
    df["implied_quote_token_usd_tvl"] = df["pair_liquidity_usd"] - df["base_token_usd_tvl"]
    df["base_token_price"] = df["pair_priceUsd"]

    df["readable_name"] = df.apply(
        lambda row: f"{row['pairAddress'][:6]}  ({row['pair_quoteToken_symbol']} /  {row['pair_baseToken_symbol']}) {row['pair_dexId']}",
        axis=1,
    )
    return df


df = await fetch_sonic_tvl()
df

Checking 3 tokens in this hop. 0
Checking 19 tokens in this hop. 1


,pairAddress,schemaVersion,pairs,pair_chainId,pair_dexId,pair_url,pair_pairAddress,pair_labels,pair_priceNative,pair_priceUsd,...,pair_info_imageUrl,pair_info_header,pair_info_openGraph,pair_info_websites,pair_info_socials,pair_priceChange_m5,base_token_usd_tvl,implied_quote_token_usd_tvl,base_token_price,readable_name
1,0x63a66Dd60b0F2812249802477adA8a890A030Eca,1.0.0,"[{'chainId': 'sonic', 'dexId': 'swapx', 'url':...",sonic,swapx,https://dexscreener.com/sonic/0x63a66dd60b0f28...,0x63a66Dd60b0F2812249802477adA8a890A030Eca,[v3],1.001487,0.9996,...,NaN,NaN,NaN,NaN,NaN,NaN,183168.703200,104412.506800,0.9996,0x63a6 (scUSD / frxUSD) swapx
2,0x7d709a567BA2fdBbB92E94E5fE74b9cbbc590835,1.0.0,"[{'chainId': 'sonic', 'dexId': 'swapx', 'url':...",sonic,swapx,https://dexscreener.com/sonic/0x7d709a567ba2fd...,0x7d709a567BA2fdBbB92E94E5fE74b9cbbc590835,[v3],1.1439,1.14,...,NaN,NaN,NaN,NaN,NaN,NaN,8927.533116,322443.126884,1.14,0x7d70 (frxUSD / sfrxUSD) swapx
4,0x2C13383855377faf5A562F1AeF47E4be7A0f12Ac,1.0.0,"[{'chainId': 'sonic', 'dexId': 'shadow-exchang...",sonic,shadow-exchange,https://dexscreener.com/sonic/0x2c13383855377f...,0x2C13383855377faf5A562F1AeF47E4be7A0f12Ac,NaN,0.9982,0.9982,...,NaN,NaN,NaN,NaN,NaN,NaN,854058.921800,229965.698200,0.9982,0x2C13 (USDC / scUSD) shadow-exchange
5,0x25ca5451CD5a50AB1d324B5E64F32C0799661891,1.0.0,"[{'chainId': 'sonic', 'dexId': 'beets', 'url':...",sonic,beets,https://dexscreener.com/sonic/0x25ca5451cd5a50...,0x25ca5451CD5a50AB1d324B5E64F32C0799661891,NaN,2.6096,1.0028,...,NaN,NaN,NaN,NaN,NaN,NaN,255324.913600,595840.826400,1.0028,0x25ca (stS / scUSD) beets
6,0x5015643B8dB50A1cB779A3e134176237d14ca67a,1.0.0,"[{'chainId': 'sonic', 'dexId': 'metropolis', '...",sonic,metropolis,https://dexscreener.com/sonic/0x5015643b8db50a...,0x5015643B8dB50A1cB779A3e134176237d14ca67a,[dlmm],1.001801,1.0018,...,NaN,NaN,NaN,NaN,NaN,NaN,16200.107800,134276.812200,1.0018,0x5015 (scUSD / USDC) metropolis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0xD7202D441778714fFE7c59879aAd163ABd91AD82,1.0.0,"[{'chainId': 'sonic', 'dexId': 'shadow-exchang...",sonic,shadow-exchange,https://dexscreener.com/sonic/0xd7202d44177871...,0xD7202D441778714fFE7c59879aAd163ABd91AD82,[v2],0.000008471,0.03145,...,https://dd.dexscreener.com/ds-data/tokens/soni...,https://dd.dexscreener.com/ds-data/tokens/soni...,https://cdn.dexscreener.com/token-images/og/so...,"[{'label': 'Website', 'url': 'https://sildao.f...","[{'type': 'twitter', 'url': 'https://x.com/sil...",-0.03,443319.860450,443597.539550,0.03145,0xD720 (WETH / SILO) shadow-exchange
74,0x58AaAa07AB256147d53FB0C9223be827EE11db03,1.0.0,"[{'chainId': 'sonic', 'dexId': 'wagmi', 'url':...",sonic,wagmi,https://dexscreener.com/sonic/0x58aaaa07ab2561...,0x58AaAa07AB256147d53FB0C9223be827EE11db03,NaN,9916.8407,3711.87,...,NaN,NaN,NaN,NaN,NaN,NaN,200556.864581,204310.315419,3711.87,0x58Aa (wS / WETH) wagmi
75,0xF58fC088C33aD46113940173cB0da3Dd08c4AA88,1.0.0,"[{'chainId': 'sonic', 'dexId': 'swapx', 'url':...",sonic,swapx,https://dexscreener.com/sonic/0xf58fc088c33ad4...,0xF58fC088C33aD46113940173cB0da3Dd08c4AA88,[v3],9929.09835,3738.30,...,NaN,NaN,NaN,NaN,NaN,NaN,230823.202650,238089.157350,3738.30,0xF58f (wS / WETH) swapx
76,0x21043D7Ad92d9e7bC45C055AF29771E37307B111,1.0.0,"[{'chainId': 'sonic', 'dexId': '0xcb2436774C3e...",sonic,0xcb2436774C3e191c85056d248EF4260ce5f27A9D,https://dexscreener.com/sonic/0x21043d7ad92d9e...,0x21043D7Ad92d9e7bC45C055AF29771E37307B111,NaN,9922.07733,3735.66,...,NaN,NaN,NaN,NaN,NaN,NaN,106335.935466,184323.354534,3735.66,0x2104 (wS / WETH) 0xcb2436774C3e191c85056d2...


In [9]:
df["pairAddress"]

1     0x63a66Dd60b0F2812249802477adA8a890A030Eca
2     0x7d709a567BA2fdBbB92E94E5fE74b9cbbc590835
4     0x2C13383855377faf5A562F1AeF47E4be7A0f12Ac
5     0x25ca5451CD5a50AB1d324B5E64F32C0799661891
6     0x5015643B8dB50A1cB779A3e134176237d14ca67a
                         ...                    
73    0xD7202D441778714fFE7c59879aAd163ABd91AD82
74    0x58AaAa07AB256147d53FB0C9223be827EE11db03
75    0xF58fC088C33aD46113940173cB0da3Dd08c4AA88
76    0x21043D7Ad92d9e7bC45C055AF29771E37307B111
79    0x0d13400CC7c46D77a43957fE614ba58C827dfde6
Name: pairAddress, Length: 68, dtype: object

In [6]:
from dataclasses import dataclass
from itertools import chain
import pandas as pd


@dataclass
class PoolNode:
    address: str
    readable_name: str
    symbols_to_usd_tvls: dict[str, float]


def df_to_pool_nodes(df: pd.DataFrame) -> list[PoolNode]:

    def row_to_node(row: pd.Series) -> PoolNode:
        symbols_to_usd_tvls = {
            row["pair_baseToken_symbol"]: round(row["base_token_usd_tvl"], 2),
            row["pair_quoteToken_symbol"]: round(row["implied_quote_token_usd_tvl"], 2),
        }
        return PoolNode(
            address=row["pairAddress"],
            readable_name=row["readable_name"],
            symbols_to_usd_tvls=symbols_to_usd_tvls,
        )

    # apply the conversion row‑by‑row and return as a plain list
    return df.apply(row_to_node, axis=1).tolist()


def find_one_hop_paths(pools: list[PoolNode], start_token: str, end_token: str) -> list[list[PoolNode]]:
    """
    Find all one-hop paths from start_token to end_token in the given pools.
    Returns a list of lists, where each inner list is a path of PoolNodes.
    """
    paths = []
    for pool in pools:
        if start_token in pool.symbols_to_usd_tvls and end_token in pool.symbols_to_usd_tvls:
            paths.append([pool])
    return paths


def find_two_hop_paths(pools: list[PoolNode], start_token: str, end_token: str) -> list[list[PoolNode]]:
    """
    Find all two-hop paths from start_token to end_token in the given pools.
    Returns a list of lists, where each inner list is a path of PoolNodes.
    """
    paths = []
    for start_pool in pools:
        if start_token in start_pool.symbols_to_usd_tvls and end_token not in start_pool.symbols_to_usd_tvls:
            # this pool can be the first hop
            other_token = next(iter(start_pool.symbols_to_usd_tvls.keys() - {start_token}))

            for second_pool in pools:
                if (
                    (second_pool.address != start_pool.address)
                    and (end_token in second_pool.symbols_to_usd_tvls)
                    and (other_token in second_pool.symbols_to_usd_tvls)
                ):
                    paths.append([start_pool, second_pool])

    return paths


# note need a recursive solution for n hops
# add later 2 hops good enough for now


pools = df_to_pool_nodes(df)
all_tokens = [list(p.symbols_to_usd_tvls.keys()) for p in pools]
token_universe = set(chain.from_iterable(all_tokens))


start_token = "scUSD"
end_token = "USDC"
one_hop_paths = find_one_hop_paths(pools, start_token, end_token)
two_hop_paths = find_two_hop_paths(pools, start_token, end_token)


def pretty_print_path(path: list[PoolNode], start_token, end_token) -> None:
    """
    Pretty print a single path of PoolNodes.
    """

    print(f'{len(path)}-hop: Path from "{start_token}" to "{end_token}":')

    # total exit liqudity

    _local_start_token = start_token
    _next_token = None
    exit_liquidity = None

    for i, pool in enumerate(path):
        token_a, token_b = pool.symbols_to_usd_tvls.keys()
        if i == 0:
            # in this case
            if token_a == start_token:
                _next_token = token_b
            else:
                _next_token = token_a
        else:
            _local_start_token = _next_token
            if _local_start_token == token_a:
                _next_token = token_b
            else:
                _next_token = token_a

        print(f"step [{i}]  in {pool}")

        print(f"swapping {_local_start_token} -> {_next_token}")
        value_in_other_token_here = pool.symbols_to_usd_tvls[_next_token]
        print(f"at most we can get out {value_in_other_token_here:.2f} in this step")

        if exit_liquidity is None:
            exit_liquidity = value_in_other_token_here
        else:
            exit_liquidity = min(exit_liquidity, value_in_other_token_here)
        print(f"{exit_liquidity} USD exit liquidity so far")
        print("")

    print("")


for path in [*one_hop_paths, *two_hop_paths]:
    pretty_print_path(path, start_token, end_token)

1-hop: Path from "scUSD" to "USDC":
step [0]  in PoolNode(address='0x2C13383855377faf5A562F1AeF47E4be7A0f12Ac', readable_name='0x2C13  (USDC /  scUSD) shadow-exchange', symbols_to_usd_tvls={'scUSD': 789496.16, 'USDC': 390374.64})
swapping scUSD -> USDC
at most we can get out 390374.64 in this step
390374.64 USD exit liquidity so far


1-hop: Path from "scUSD" to "USDC":
step [0]  in PoolNode(address='0xDd35c88B1754879EF86BBF3A24F81fCCA5Eb6B5D', readable_name='0xDd35  (USDC /  scUSD) swapx', symbols_to_usd_tvls={'scUSD': 230582.73, 'USDC': 7304.7})
swapping scUSD -> USDC
at most we can get out 7304.70 in this step
7304.7 USD exit liquidity so far


1-hop: Path from "scUSD" to "USDC":
step [0]  in PoolNode(address='0x5015643B8dB50A1cB779A3e134176237d14ca67a', readable_name='0x5015  (scUSD /  USDC) metropolis', symbols_to_usd_tvls={'USDC': 27272.41, 'scUSD': 123738.57})
swapping scUSD -> USDC
at most we can get out 27272.41 in this step
27272.41 USD exit liquidity so far


2-hop: Path fro

In [ ]:
sc_pools_df = pd.DataFrame([p for p in pools if p.symbols_to_usd_tvls.get("scUSD")])

sc_pools_df["tokens"] = sc_pools_df.apply(lambda row: list(row["symbols_to_usd_tvls"].keys()), axis=1)

# get the other token key in pools
sc_pools_df["other_token"] = sc_pools_df["tokens"].apply(
    lambda x: [t for t in x if t != "scUSD"][0] if len(x) > 1 else None
)
sc_pools_df["other_token_tvl"] = sc_pools_df.apply(
    lambda row: row["symbols_to_usd_tvls"].get(row["other_token"], 0), axis=1
)
sc_pools_df

,address,readable_name,symbols_to_usd_tvls,tokens,other_token,other_token_tvl
0,0x63a66Dd60b0F2812249802477adA8a890A030Eca,0x63a6 (scUSD / frxUSD) swapx,"{'frxUSD': 213827.76, 'scUSD': 55659.69}","[frxUSD, scUSD]",frxUSD,213827.76
1,0x2C13383855377faf5A562F1AeF47E4be7A0f12Ac,0x2C13 (USDC / scUSD) shadow-exchange,"{'scUSD': 328089.58, 'USDC': 960902.36}","[scUSD, USDC]",USDC,960902.36
2,0xDd35c88B1754879EF86BBF3A24F81fCCA5Eb6B5D,0xDd35 (USDC / scUSD) swapx,"{'scUSD': 7580.28, 'USDC': 244831.44}","[scUSD, USDC]",USDC,244831.44
3,0x5015643B8dB50A1cB779A3e134176237d14ca67a,0x5015 (scUSD / USDC) metropolis,"{'USDC': 51585.39, 'scUSD': 93273.86}","[USDC, scUSD]",USDC,51585.39
4,0x25ca5451CD5a50AB1d324B5E64F32C0799661891,0x25ca (stS / scUSD) beets,"{'scUSD': 258943.92, 'stS': 604238.02}","[scUSD, stS]",stS,604238.02
5,0xEd08f5caD599E7F523d6B3FD598005B43aA003bb,0xEd08 (scUSD / wstkscUSD) swapx,"{'wstkscUSD': 50390.92, 'scUSD': 61921.03}","[wstkscUSD, scUSD]",wstkscUSD,50390.92
6,0xC4D8199ad20a808819da3fd86b859B11fFe886B4,0xC4D8 (scUSD / scETH) beets,"{'scETH': 0.28, 'scUSD': 150208.06}","[scETH, scUSD]",scETH,0.28
7,0xc5AB8d98F959453E416b2F15848a02CC99bc695e,0xc5AB (scUSD / BRUSH) beets,"{'BRUSH': 139232.6, 'scUSD': 34872.9}","[BRUSH, scUSD]",BRUSH,139232.60
8,0xd501FB4eeb19CDD8a1B41e33226bDd1EF04495Ba,0xd501 (scUSD / msUSD) shadow-exchange,"{'msUSD': 68571.23, 'scUSD': 58930.52}","[msUSD, scUSD]",msUSD,68571.23


In [ ]:
sc_pools_df["other_token_tvl"].sum().round()

np.float64(2333580.0)

In [ ]:
# I'm thinking about this in terms of differentlly sized pipes,

# (in pool 1 we can only get out 1M))

In [ ]:
pd.DataFrame(pools)

,address,readable_name,symbols_to_usd_tvls
0,0x63a66Dd60b0F2812249802477adA8a890A030Eca,0x63a6 (scUSD / frxUSD) swapx,"{'frxUSD': 213827.76, 'scUSD': 55659.69}"
1,0x7d709a567BA2fdBbB92E94E5fE74b9cbbc590835,0x7d70 (frxUSD / sfrxUSD) swapx,"{'sfrxUSD': 5799.18, 'frxUSD': 325803.87}"
2,0x324963c267C354c7660Ce8CA3F5f167E05649970,0x3249 (USDC / wS) shadow-exchange,"{'wS': 1958938.78, 'USDC': 1595692.99}"
3,0x6Fb30F3FCB864D49cdff15061ed5c6ADFEE40B40,0x6Fb3 (USDC / WETH) shadow-exchange,"{'WETH': 872790.74, 'USDC': 2179940.62}"
4,0x2C13383855377faf5A562F1AeF47E4be7A0f12Ac,0x2C13 (USDC / scUSD) shadow-exchange,"{'scUSD': 328089.58, 'USDC': 960902.36}"
...,...,...,...
60,0xdEE813F080f9128e52E38E9Ffef8B997F9544332,0xdEE8 (HLP0 / xUSD) shadow-exchange,"{'xUSD': 1977527.28, 'HLP0': 1993727.37}"
61,0xD7202D441778714fFE7c59879aAd163ABd91AD82,0xD720 (WETH / SILO) shadow-exchange,"{'SILO': 475990.97, 'WETH': 476034.91}"
62,0x58AaAa07AB256147d53FB0C9223be827EE11db03,0x58Aa (wS / WETH) wagmi,"{'WETH': 155293.4, 'wS': 154615.45}"
63,0x21043D7Ad92d9e7bC45C055AF29771E37307B111,0x2104 (wS / WETH) 0xcb2436774C3e191c85056d2...,"{'WETH': 113766.63, 'wS': 169581.8}"


{'sfrxUSD', 'wstkscUSD', 'smsUSD', 'WETH', 'stS', 'Anon', 'BEETS', 'wmetaUSD', 'xUSD', 'x33', 'USDT', 'stkscETH', 'scETH', 'HLP0', 'xSILO', 'RZR', 'BeetsFragmentsS1', 'BRUSH', 'bUSDC-20', 'USDC', 'frxUSD', 'fBOMB', 'OS', 'scUSD', 'SHADOW', 'scBTC', 'SWPx', 'LBTC', 'wS', 'SILO', 'msUSD', 'WBTC'}


In [ ]:
two_hop_paths = find_two_hop_paths(pools, "Anon", "USDC")
two_hop_paths

[[PoolNode(address='0x297000941c155962B04760EcAeAA422757F20bc1', readable_name='wagmi - Anon/wS', symbols_to_usd_tvls={'Anon': 317766.08, 'wS': 55351.44}),
  PoolNode(address='0x324963c267C354c7660Ce8CA3F5f167E05649970', readable_name='shadow-exchange - wS/USDC', symbols_to_usd_tvls={'wS': 2146726.77, 'USDC': 1344792.3})],
 [PoolNode(address='0x297000941c155962B04760EcAeAA422757F20bc1', readable_name='wagmi - Anon/wS', symbols_to_usd_tvls={'Anon': 317766.08, 'wS': 55351.44}),
  PoolNode(address='0x5C4B7d607aAF7B5CDE9F09b5F03Cf3b5c923AEEa', readable_name='swapx - wS/USDC', symbols_to_usd_tvls={'wS': 1016355.74, 'USDC': 500578.58})],
 [PoolNode(address='0x297000941c155962B04760EcAeAA422757F20bc1', readable_name='wagmi - Anon/wS', symbols_to_usd_tvls={'Anon': 317766.08, 'wS': 55351.44}),
  PoolNode(address='0x25f746bB206041Ed8dA6F08Ed1D32454A5856D37', readable_name='wagmi - wS/USDC', symbols_to_usd_tvls={'wS': 540826.99, 'USDC': 120543.96})],
 [PoolNode(address='0x297000941c155962B04760Ec

In [ ]:
def pretty_print_path(path: list[PoolNode]) -> None:
    """
    Pretty print a single path of PoolNodes.
    """
    names = " -> ".join([p.readable_name for p in path])
    print(f"{len(path)}‑hop: {names}")


for path in two_hop_paths:
    pretty_print_path(path)

2‑hop: wagmi - Anon/wS -> shadow-exchange - wS/USDC
2‑hop: wagmi - Anon/wS -> swapx - wS/USDC
2‑hop: wagmi - Anon/wS -> wagmi - wS/USDC
2‑hop: wagmi - Anon/wS -> silverswap - wS/USDC
2‑hop: wagmi - Anon/wS -> wagmi - wS/USDC
2‑hop: wagmi - Anon/wS -> solidlycom - wS/USDC


In [ ]:
def pretty_print_paths(paths: list[list[PoolNode]], start_token: str, end_token: str) -> None:
    """
    Nicely print 1‑hop and 2‑hop paths from start_token to end_token.
    """
    if not paths:
        print(f"No paths found from {start_token} → {end_token}")
        return

    for i, path in enumerate(paths, 1):
        if len(path) == 1:
            p = path[0]
            print(f"[{i}] Direct: {p.readable_name} ({start_token} ↔ {end_token})")
        elif len(path) == 2:
            p1, p2 = path
            # find the intermediate symbol
            syms1 = set(p1.symbols_to_usd_tvls)
            syms2 = set(p2.symbols_to_usd_tvls)
            inter = (syms1 & syms2) - {start_token, end_token}
            inter_sym = inter.pop() if inter else "<?>"
            print(f"[{i}] 2‑hop:")
            print(f"     1. {p1.readable_name} ({start_token} ↔ {inter_sym})")
            print(f"     2. {p2.readable_name} ({inter_sym} ↔ {end_token})")
        else:
            # just in case you ever extend to >2 hops
            names = " → ".join(p.readable_name for p in path)
            print(f"[{i}] {len(path)}‑hop: {names}")
        print()


find_two_hop_paths(two_hop_paths, "Anon", "USDC")

AttributeError: 'list' object has no attribute 'symbols_to_usd_tvls'

In [ ]:
sc_pools = [p for p in pools if "scUSD" in p.symbols_to_usd_tvls]
sc_pools

[PoolNode(address='0x63a66Dd60b0F2812249802477adA8a890A030Eca', readable_name='swapx - frxUSD/scUSD', symbols_to_usd_tvls={'frxUSD': 214771.95, 'scUSD': 54716.21}),
 PoolNode(address='0x2C13383855377faf5A562F1AeF47E4be7A0f12Ac', readable_name='shadow-exchange - scUSD/USDC', symbols_to_usd_tvls={'scUSD': 329640.03, 'USDC': 959350.79}),
 PoolNode(address='0xDd35c88B1754879EF86BBF3A24F81fCCA5Eb6B5D', readable_name='swapx - scUSD/USDC', symbols_to_usd_tvls={'scUSD': 8019.32, 'USDC': 244392.13}),
 PoolNode(address='0x5015643B8dB50A1cB779A3e134176237d14ca67a', readable_name='metropolis - USDC/scUSD', symbols_to_usd_tvls={'USDC': 52612.31, 'scUSD': 92246.57}),
 PoolNode(address='0x25ca5451CD5a50AB1d324B5E64F32C0799661891', readable_name='beets - scUSD/stS', symbols_to_usd_tvls={'scUSD': 257374.32, 'stS': 600622.13}),
 PoolNode(address='0xEd08f5caD599E7F523d6B3FD598005B43aA003bb', readable_name='swapx - wstkscUSD/scUSD', symbols_to_usd_tvls={'wstkscUSD': 52187.71, 'scUSD': 60120.57}),
 PoolNod

In [ ]:
sc_pools = [p for p in pools if "frxUSD" in p.symbols_to_usd_tvls]
sc_pools

[PoolNode(address='0x63a66Dd60b0F2812249802477adA8a890A030Eca', readable_name='swapx - frxUSD/scUSD', symbols_to_usd_tvls={'frxUSD': 214771.95, 'scUSD': 54716.21}),
 PoolNode(address='0x7d709a567BA2fdBbB92E94E5fE74b9cbbc590835', readable_name='swapx - sfrxUSD/frxUSD', symbols_to_usd_tvls={'sfrxUSD': 5296.54, 'frxUSD': 325168.29})]

In [ ]:
# there is a good amount of data cleaning to do here, whitelist tokens

,pairAddress,base_token_usd_tvl,base_token_price,implied_quote_token_usd_tvl,pair_liquidity_usd,pair_baseToken_symbol,pair_quoteToken_symbol,readable_name
1,0x63a66Dd60b0F2812249802477adA8a890A030Eca,214771.95,1.00020,54716.21,269488.16,frxUSD,scUSD,swapx - frxUSD/scUSD
2,0x7d709a567BA2fdBbB92E94E5fE74b9cbbc590835,5296.54,1.14,325168.29,330464.83,sfrxUSD,frxUSD,swapx - sfrxUSD/frxUSD
4,0x324963c267C354c7660Ce8CA3F5f167E05649970,2146726.77,0.3722,1344792.30,3491519.07,wS,USDC,shadow-exchange - wS/USDC
5,0x6Fb30F3FCB864D49cdff15061ed5c6ADFEE40B40,1017729.15,3496.53,2019389.63,3037118.78,WETH,USDC,shadow-exchange - WETH/USDC
6,0x5C4B7d607aAF7B5CDE9F09b5F03Cf3b5c923AEEa,1016355.74,0.3731,500578.58,1516934.32,wS,USDC,swapx - wS/USDC
...,...,...,...,...,...,...,...,...
75,0xdEE813F080f9128e52E38E9Ffef8B997F9544332,1975777.80,1.22,1990516.10,3966293.90,xUSD,HLP0,shadow-exchange - xUSD/HLP0
77,0xD7202D441778714fFE7c59879aAd163ABd91AD82,467677.67,0.03722,467858.40,935536.07,SILO,WETH,shadow-exchange - SILO/WETH
78,0x58AaAa07AB256147d53FB0C9223be827EE11db03,146683.25,3487.56,148595.88,295279.13,WETH,wS,wagmi - WETH/wS
79,0x21043D7Ad92d9e7bC45C055AF29771E37307B111,110521.33,3483.62,166926.28,277447.61,WETH,wS,0xcb2436774C3e191c85056d248EF4260ce5f27A9D - W...


In [ ]:
# 0x8BC2f9e725cbB07c338df4e77c82190119ddd823
# off by maybe 10%

#

In [ ]:
# does it mean, liqudiity in terms

In [ ]:
# "liquidity": {
#   "usd": 1_000_000,
#   "base": 800,
#   "quote": 500_000
# }
# usd (number)
# Total dollar‑value of assets locked in the pool.

# base (number)
# Amount of the base token currently in the pool.

# quote (number)
# Amount of the quote token currently in the pool.


# that is the claim

In [ ]:
df["base_token_to_usd"] = df["pair_priceUsd"].astype(float)
df["base_in_terms_of_quote"] = 1 / df["quote_in_terms_of_base"]

df["naive_usd_tvl"] = df["pair_liquidity_quote"] + df["pair_liquidity_base"]

# bad
# df["usd_tvl_adjusting_for_prices"] = (df["pair_liquidity_quote"] / df["base_in_terms_of_quote"]) + (
#     df["pair_liquidity_base"] / df["base_in_terms_of_quote"]
# )


# df["usd_tvl_adjusting_for_prices"] = (df["pair_liquidity_quote"] * df["quote_in_terms_of_base"]) + (
#     df["pair_liquidity_base"] * df["base_in_terms_of_quote"]
# )

# better still not good enough
df["usd_tvl_adjusting_for_prices"] = (df["pair_liquidity_quote"] * df["base_in_terms_of_quote"]) + (
    df["pair_liquidity_base"] * df["quote_in_terms_of_base"]
)

df["usd_tvl_adjusting_for_prices"] = (df["pair_liquidity_quote"] * 1) + (df["pair_liquidity_base"] * 1)


df["readable_name"] = df["pair_dexId"] + " - " + df["pair_baseToken_symbol"] + "/" + df["pair_quoteToken_symbol"]

local_df = df.drop_duplicates(subset=["readable_name"]).copy()

local_df["abs_percent_difference"] = (
    (local_df["usd_tvl_adjusting_for_prices"] - local_df["pair_liquidity_usd"]).abs()
    / local_df["pair_liquidity_usd"]
    * 100
)
print(local_df["abs_percent_difference"].describe().round())
local_df[
    [
        "quote_in_terms_of_base",
        "base_in_terms_of_quote",
        "usd_tvl_adjusting_for_prices",
        "readable_name",
        "pair_liquidity_usd",
        "naive_usd_tvl",
        "pair_baseToken_symbol",
        "pair_quoteToken_symbol",
        "abs_percent_difference",
    ]
].round(5)

KeyError: 'quote_in_terms_of_base'

In [ ]:
break

In [ ]:
pivoted = (
    local_df.melt(
        id_vars="readable_name",
        value_vars=["pair_liquidity_usd", "naive_usd_tvl", "usd_tvl_adjusting_for_prices"],
        var_name="metric",
        value_name="value",
    )
    .drop_duplicates()
    .pivot(index="metric", columns="readable_name", values="value")
).T


pivoted["percent_difference"] = (
    (pivoted["naive_usd_tvl"] - pivoted["pair_liquidity_usd"]) / pivoted["pair_liquidity_usd"] * 100
).abs()
print(pivoted)

pivoted.round().sort_values("percent_difference", ascending=False)

metric,naive_usd_tvl,pair_liquidity_usd,usd_tvl_adjusting_for_prices,percent_difference
readable_name,,,,
swapx - bUSDC-20/wstkscUSD,308329077.0,532541.0,213328100.0,57798.0
beets - BRUSH/scUSD,5807456.0,174251.0,1580818.0,3233.0
shadow-exchange - xSILO/BRUSH,3605481.0,118067.0,46041819.0,2954.0
shadow-exchange - BRUSH/wS,15585827.0,568617.0,23929874.0,2641.0
beets - BEETS/stS,101232642.0,3939572.0,67879880.0,2470.0
...,...,...,...,...
swapx - wmetaUSD/wstkscUSD,188420.0,189091.0,188934.0,0.0
swapx - wstkscUSD/scUSD,111839.0,112308.0,111760.0,0.0
shadow-exchange - HLP0/USDC,674294.0,674894.0,672323.0,0.0


In [ ]:
pivoted["percent_difference"] = (
    (pivoted["naive_usd_tvl"] - pivoted["pair_liquidity_usd"]) / pivoted["pair_liquidity_usd"] * 100
).abs()
pivoted.round().sort_values("percent_difference", ascending=False)

In [ ]:
df.columns

Index(['pairAddress', 'schemaVersion', 'pairs', 'pair_chainId', 'pair_dexId',
       'pair_url', 'pair_pairAddress', 'pair_labels', 'pair_priceNative',
       'pair_priceUsd', 'pair_fdv', 'pair_marketCap', 'pair_baseToken_address',
       'pair_baseToken_name', 'pair_baseToken_symbol',
       'pair_quoteToken_address', 'pair_quoteToken_name',
       'pair_quoteToken_symbol', 'pair_txns_m5_buys', 'pair_txns_m5_sells',
       'pair_txns_h1_buys', 'pair_txns_h1_sells', 'pair_txns_h6_buys',
       'pair_txns_h6_sells', 'pair_txns_h24_buys', 'pair_txns_h24_sells',
       'pair_volume_h24', 'pair_volume_h6', 'pair_volume_h1', 'pair_volume_m5',
       'pair_priceChange_h24', 'pair_liquidity_usd', 'pair_liquidity_base',
       'pair_liquidity_quote', 'pair_pairCreatedAt', 'pair_priceChange_m5',
       'pair_priceChange_h1', 'pair_priceChange_h6', 'pair_info_imageUrl',
       'pair_info_header', 'pair_info_openGraph', 'pair_info_websites',
       'pair_info_socials', 'quote_in_terms_of_base', '

In [ ]:
df[["readable_name", "pairAddress"]].value_counts()

readable_name                                           pairAddress                               
0xcb2436774C3e191c85056d248EF4260ce5f27A9D - WETH/USDC  0xCfD41dF89D060b72eBDd50d65f9021e4457C477e    1
swapx - frxUSD/scUSD                                    0x63a66Dd60b0F2812249802477adA8a890A030Eca    1
shadow-exchange - wS/USDC                               0x324963c267C354c7660Ce8CA3F5f167E05649970    1
shadow-exchange - x33/wS                                0xCF0A945ed064Ac32Ec8D178F865F650a9f9A2fDb    1
shadow-exchange - xSILO/BRUSH                           0x27ff519BbEac6eB307D6e95758cbfBf808A7827b    1
                                                                                                     ..
shadow-exchange - WETH/USDC                             0x6Fb30F3FCB864D49cdff15061ed5c6ADFEE40B40    1
shadow-exchange - WETH/wS                               0xB6d9B069F6B96A507243d501d1a23b3fCCFC85d3    1
shadow-exchange - msUSD/scUSD                           0xd501FB4eeb1

In [ ]:
import numpy as np


def get_tokemak_portion_ownership_by_pool(full_df: pd.DataFrame) -> dict:
    # stub
    np.random.seed(42)
    choices = [0, 0.1]
    portion_ownership = {a: float(np.random.choice(choices)) for a in full_df["address"].tolist()}

    # portion_ownership = get_tokemak_portion_ownership_by_pool(full_df)

    # full_df["our_portion_ownership"] = full_df["address"].map(portion_ownership)
    # full_df["scaled_base_token_tvl"] = full_df["base_token_usd_tvl"] * (1 - full_df["our_portion_ownership"])
    # full_df["scaled_quote_token_tvl"] = full_df["quote_token_tvl"] * (1 - full_df["our_portion_ownership"])
    return portion_ownership